In [13]:
# Set cutoff variable here
cutoff = 0.5

In [14]:
# import packages
import pandas as pd
import numpy as np
import statsmodels.api as sm
# import data
sites = pd.read_table("../data/eco_data/eco_sites.txt")
counts = pd.read_table("../data/eco_data/species_counts.txt")

# Add has_mega column to determine which sites have Megalaima
is_mega = counts.columns.str.contains('Megalaima')
sites['has_mega'] = counts.loc[:,is_mega].sum(axis=1) > 0

# Create model
to_model = sites.loc[:,['has_mega','Elevation']]
to_model = sm.add_constant(to_model)
y = to_model['has_mega']
X = to_model.drop(['has_mega'],axis=1)
lr_mod = sm.Logit(y, X).fit()
pred = lr_mod.predict(X)

# Find total0s and total1s (total has_mega and total !has_mega)
total0s = (sites['has_mega'] == 0).astype(int).sum()
total1s = (sites['has_mega'] == 1).astype(int).sum()

# Use the cutoff to generate has_mega_preds and add it to the df
sites['has_mega_preds'] = (pred > cutoff).astype(int)

# Find the number of true positives (real is 1, pred is 1) 
true_positives = ((sites['has_mega'] == 1) & (sites['has_mega_preds'] == 1)).astype(int).sum()

# Find the number of false positives (real is 0, pred is 1)
false_positives = ((sites['has_mega'] == 0) & (sites['has_mega_preds'] == 1)).astype(int).sum()

# Calculate/Print the true positive rate (tpr) and false positive rate (fpr)
tpr = true_positives / total1s
fpr = false_positives / total0s
print(tpr, fpr)


Optimization terminated successfully.
         Current function value: 0.329815
         Iterations 7
0.8235294117647058 0.09523809523809523
